In [8]:
import pandas as pd
import numpy as np
import reverse_geocoder as rg

In [9]:
# calls = pd.read_csv("requetes311.csv")

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,3,4,5,6,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [327]:
calls.shape

(1371320, 10)

In [328]:
calls = calls[pd.notnull(calls['ARRONDISSEMENT'])]
calls.shape

(1371320, 10)

In [329]:
calls[:10]

,NATURE,ACTI_NOM,ARRONDISSEMENT,PROVENANCE_TELEPHONE,PROVENANCE_COURRIEL,PROVENANCE_PERSONNE,PROVENANCE_COURRIER,UNITE_RESP_PARENT,Date,Time
1420351,Requete,Taxes foncières,Ville de Montréal,1,0,0,0,SERVICE DES FINANCES,01,10
1420352,Requete,*Capital humain - Ressources humaines,Ville de Montréal,1,0,0,0,SERVICE DE LA CONCERTATION DES ARRONDISSEMENTS,01,13
1420353,Requete,IRP - Villes non reliées au GDC,Montréal-Est,1,0,0,0,SECTION CENTRE DE SERVICES 311,02,21
1420354,Requete,IRP - Villes non reliées au GDC,Montréal-Est,0,0,1,0,SECTION CENTRE DE SERVICES 311,02,09
1420355,Requete,Taxes foncières,Montréal-Est,1,0,0,0,SERVICE DES FINANCES,02,16
1420356,Requete,IRP - Villes non reliées au GDC,Montréal-Est,1,0,0,0,SECTION CENTRE DE SERVICES 311,02,08
1420357,Requete,IRP - Villes non reliées au GDC,Mont-Royal,1,0,0,0,SECTION CENTRE DE SERVICES 311,03,07
1420358,Requete,Cour municipale,Ville de Montréal,0,0,1,0,DIRECTION DES AFFAIRES PENALES ET CRIMINELLES,03,15
1420359,Requete,Direction - Évaluation,Ville de Montréal,1,0,0,0,SERVICE DES AFFAIRES JURIDIQUES,03,15
1420360,Requete,Communications corporatives,Ville de Montréal,0,1,0,0,SERVICE DE L EXPÉRIENCE CITOYENNE ET DES COMMU...,03,14


In [330]:
del calls['ID_UNIQUE']
del calls['TYPE_LIEU_INTERV']
del calls['RUE']
del calls['RUE_INTERSECTION1']
del calls['RUE_INTERSECTION2']

KeyError: 'ID_UNIQUE'

In [331]:
del calls['LOC_ERREUR_GDT']
del calls['ARRONDISSEMENT_GEO']
del calls['PROVENANCE_TELECOPIEUR']
del calls['PROVENANCE_INSTANCE']
del calls['PROVENANCE_MOBILE']
del calls['PROVENANCE_MEDIASOCIAUX']
del calls['PROVENANCE_SITEINTERNET']

KeyError: 'LOC_ERREUR_GDT'

In [14]:
del calls['LIN_CODE_POSTAL']
del calls['PROVENANCE_ORIGINALE']

In [15]:
del calls['LOC_LONG']
del calls['LOC_LAT']
del calls['LOC_X']
del calls['LOC_Y']

In [414]:
calls.shape

(1371320, 10)

In [333]:
calls['Date'], calls['Time'] = calls['DDS_DATE_CREATION'].str.split('T', 1).str

KeyError: 'DDS_DATE_CREATION'

In [18]:
calls['Time'] = (calls['DDS_DATE_CREATION'].str[11:13])
calls['Date'] = (calls['DDS_DATE_CREATION'].str[5:7])
del calls['DDS_DATE_CREATION']

In [ ]:
calls.to_csv("calls.csv")

In [415]:
data = calls.values
data.shape

(1371320, 10)

In [416]:
print((calls['NATURE'].unique().tolist()))
print(len(calls['ACTI_NOM'].unique().tolist()))
print(len(calls['ARRONDISSEMENT'].unique().tolist()))
print(len(calls['UNITE_RESP_PARENT'].unique().tolist()))
# calls.groupby('UNITE_RESP_PARENT').count()

['Requete', 'Commentaire', 'Plainte']
1524
36
50


In [417]:
def time_of_day(time):
    time = int(time)
    if(time==None):
        return "other"
    if(time >= 23 or time <= 4):
        return "nuit"
    if(time >= 5 and time <= 10):
        return "jour"
    if(time >= 11 and time <= 16):
        return "midi"
    if(time >= 17 and time <= 22):
        return "soir"

In [418]:
dummyNature = pd.get_dummies(calls['NATURE'])
calls_dummified = pd.concat([calls,dummyNature],axis=1)

In [419]:
calls_dummified.shape

(1371320, 13)

In [420]:
calls_dummified['Time'] = calls_dummified['Time'].apply(time_of_day)

In [421]:
calls_dummified.shape

(1371320, 13)

In [422]:
dummyTime = pd.get_dummies(calls_dummified['Time'])
calls_dummified = pd.concat([calls_dummified, dummyTime], axis=1)

In [423]:
calls_dummified['Month'] = calls_dummified['Date']

In [424]:
del calls_dummified['Date']
calls_dummified.head()

,NATURE,ACTI_NOM,ARRONDISSEMENT,PROVENANCE_TELEPHONE,PROVENANCE_COURRIEL,PROVENANCE_PERSONNE,PROVENANCE_COURRIER,UNITE_RESP_PARENT,Time,Commentaire,Plainte,Requete,jour,midi,nuit,soir,Month
1420351,Requete,Taxes foncières,Ville de Montréal,1,0,0,0,SERVICE DES FINANCES,jour,0,0,1,1,0,0,0,01
1420352,Requete,*Capital humain - Ressources humaines,Ville de Montréal,1,0,0,0,SERVICE DE LA CONCERTATION DES ARRONDISSEMENTS,midi,0,0,1,0,1,0,0,01
1420353,Requete,IRP - Villes non reliées au GDC,Montréal-Est,1,0,0,0,SECTION CENTRE DE SERVICES 311,soir,0,0,1,0,0,0,1,02
1420354,Requete,IRP - Villes non reliées au GDC,Montréal-Est,0,0,1,0,SECTION CENTRE DE SERVICES 311,jour,0,0,1,1,0,0,0,02
1420355,Requete,Taxes foncières,Montréal-Est,1,0,0,0,SERVICE DES FINANCES,midi,0,0,1,0,1,0,0,02


In [425]:
del calls_dummified['Time']

In [426]:
del calls_dummified['NATURE']

In [427]:
def month_of_year(time):
    time = int(time)
    if(time==None):
        return "other"
    if(time >= 8 and time <= 11):
        return "fall"
    if(time >= 4 and time <= 7):
        return "summer"
    if(time >= 12 or time <= 3):
        return "winter"

In [428]:
calls_dummified['Weather'] = calls_dummified['Month'].apply(month_of_year)

In [429]:
dummyWeather = pd.get_dummies(calls_dummified['Weather'])
calls_dummified = pd.concat([calls_dummified, dummyWeather], axis=1)

In [430]:
del calls_dummified['Month']

In [431]:
del calls_dummified['Weather']

In [432]:
calls_dummified.shape

(1371320, 17)

In [433]:
ll = calls_dummified.groupby(['ACTI_NOM'])['ACTI_NOM'].agg(
    {"code_count": len}).sort_values(
    "code_count", ascending=False).head(10)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [434]:
ll

,code_count
ACTI_NOM,
Permis animalier,176792
Collecte de branches d'arbre - Agrile du frêne,56629
Collecte des encombrants,45055
Éclairage existant - Entretien,42904
Collecte de déchets,42154
Dépôt illégal - Déchets,38100
Opération déneigement,32702
Collecte des matières recyclables,28967
Feux de circulation - Entretien,28954


In [435]:
cats = ll.index.tolist()
cats

['Permis animalier',
 "Collecte de branches d'arbre - Agrile du frêne",
 'Collecte des encombrants',
 'Éclairage existant - Entretien',
 'Collecte de déchets',
 'Dépôt illégal - Déchets',
 'Opération déneigement',
 'Collecte des matières recyclables',
 'Feux de circulation - Entretien',
 'Nid-de-poule']

In [436]:
calls_dummified.loc[~calls_dummified.ACTI_NOM.isin(cats),'ACTI_NOM']="Other"

In [437]:
calls_dummified['ACTI_NOM'].unique().tolist()

['Other',
 'Permis animalier',
 'Collecte de déchets',
 'Collecte des matières recyclables',
 'Dépôt illégal - Déchets',
 'Feux de circulation - Entretien',
 'Nid-de-poule',
 'Éclairage existant - Entretien',
 'Collecte des encombrants',
 "Collecte de branches d'arbre - Agrile du frêne",
 'Opération déneigement']

In [438]:
calls_dummified['ACTI_NOM'].value_counts()

Other                                             852426
Permis animalier                                  176792
Collecte de branches d'arbre - Agrile du frêne     56629
Collecte des encombrants                           45055
Éclairage existant - Entretien                     42904
Collecte de déchets                                42154
Dépôt illégal - Déchets                            38100
Opération déneigement                              32702
Collecte des matières recyclables                  28967
Feux de circulation - Entretien                    28954
Nid-de-poule                                       26637
Name: ACTI_NOM, dtype: int64

In [439]:
calls_dummified.shape

(1371320, 17)

In [440]:
dummyARRONDISSEMENT = pd.get_dummies(calls_dummified['ARRONDISSEMENT'])
calls_dummified = pd.concat([calls_dummified, dummyARRONDISSEMENT], axis=1)

dummyUNITE_RESP_PARENT = pd.get_dummies(calls_dummified['UNITE_RESP_PARENT'])
calls_dummified = pd.concat([calls_dummified, dummyUNITE_RESP_PARENT], axis=1)

del calls_dummified['ARRONDISSEMENT']
del calls_dummified['UNITE_RESP_PARENT']

In [441]:
list(calls_dummified.columns.values)
calls_dummified.head()

,ACTI_NOM,PROVENANCE_TELEPHONE,PROVENANCE_COURRIEL,PROVENANCE_PERSONNE,PROVENANCE_COURRIER,Commentaire,Plainte,Requete,jour,midi,...,SERVICE DES AFFAIRES JURIDIQUES,SERVICE DES FINANCES,"SERVICE DES GRANDS PARCS, DU MONT-ROYAL ET DES SPORTS",SERVICE DES INFRASTRUCTURES DU RÉSEAU ROUTIER,SERVICE DES RESSOURCES HUMAINES,ST-LAURENT,STATIONNEMENT DE MONTRÉAL - DIR. GÉN.,SUD-OUEST,VILLE-MARIE,VILLERAY - ST-MICHEL - PARC-EXTENSION
1420351,Other,1,0,0,0,0,0,1,1,0,...,0,1,0,0,0,0,0,0,0,0
1420352,Other,1,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1420353,Other,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1420354,Other,0,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1420355,Other,1,0,0,0,0,0,1,0,1,...,0,1,0,0,0,0,0,0,0,0


In [442]:
# del calls_dummified['Weather']
calls_dummified.shape

(1371320, 101)

In [443]:
dummyACTI_NOM = pd.get_dummies(calls_dummified['ACTI_NOM'])
calls_dummified = pd.concat([calls_dummified, dummyACTI_NOM], axis=1)

In [444]:
del calls_dummified['ACTI_NOM']
# list(calls_dummified.columns.values)

In [463]:
calls_dummified = calls_dummified.sample(frac=1)
dataset = calls_dummified

,PROVENANCE_TELEPHONE,PROVENANCE_COURRIEL,PROVENANCE_PERSONNE,PROVENANCE_COURRIER,Commentaire,Plainte,Requete,jour,midi,nuit,...,Collecte de déchets,Collecte des encombrants,Collecte des matières recyclables,Dépôt illégal - Déchets,Feux de circulation - Entretien,Nid-de-poule,Opération déneigement,Other,Permis animalier,Éclairage existant - Entretien
2459934,1,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1855944,0,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2263145,0,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1972163,0,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1815129,1,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [464]:
dataset.shape

(1371320, 111)

# Model Train

In [465]:
#Splitting the data into independent and dependent variables
X = dataset.iloc[:,:100].values
dataset.iloc[:,:100]

,PROVENANCE_TELEPHONE,PROVENANCE_COURRIEL,PROVENANCE_PERSONNE,PROVENANCE_COURRIER,Commentaire,Plainte,Requete,jour,midi,nuit,...,SERVICE DES AFFAIRES JURIDIQUES,SERVICE DES FINANCES,"SERVICE DES GRANDS PARCS, DU MONT-ROYAL ET DES SPORTS",SERVICE DES INFRASTRUCTURES DU RÉSEAU ROUTIER,SERVICE DES RESSOURCES HUMAINES,ST-LAURENT,STATIONNEMENT DE MONTRÉAL - DIR. GÉN.,SUD-OUEST,VILLE-MARIE,VILLERAY - ST-MICHEL - PARC-EXTENSION
2459934,1,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1855944,0,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2263145,0,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
1972163,0,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1815129,1,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1812312,1,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1914283,1,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2486672,1,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1694715,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1734031,0,0,1,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [517]:
y = dataset.iloc[:,100:].values
y.shape

(1371320, 11)

In [518]:
ll = [np.where(r==1)[0][0] for r in y]
ll = np.array(ll)

In [ ]:
np.where(y==1)[0][0]

In [498]:
X.T.shape

(1371320, 100)

In [499]:
y.T.shape

(1371320, 11)

In [492]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [519]:
y=ll

In [520]:
print(X.shape)
print(y.shape)

(1371320, 100)
(1371320,)


In [521]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)

In [522]:
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [523]:
y_pred = classifier.predict(X_test)

In [524]:
def display_results(y_val, y_pred):
    print("Accuracy % = ", metrics.accuracy_score(y_val, y_pred))

In [525]:
display_results(y_test, y_pred)

Accuracy % =  0.6985298836157863


In [526]:
metrics.f1_score(y_test, y_pred, average='weighted')

0.6364098727222846

In [527]:
from sklearn.linear_model import LogisticRegression
modelLR = LogisticRegression()
modelLR.fit(X_train, y_train)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [534]:
y_pred_LR = modelLR.predict(X_test)

In [535]:
display_results(y_test, y_pred_LR)

Accuracy % =  0.6833328471837353


In [537]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam

Using TensorFlow backend.


In [545]:
from sklearn.neural_network import MLPClassifier
modelclf = MLPClassifier()
modelclf.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [546]:
y_pred_MLP = modelclf.predict(X_test)

In [547]:
display_results(y_test, y_pred_MLP)

Accuracy % =  0.6991365983140332
